# Hente terminar
Dette er ei lita rutine for å hente ut alle formelle terminar som vert brukt i undervising. 

In [1]:
import requests
import os
import pandas as pd
tokenCanvas = os.environ['tokenCanvas']

headers = {
    'Authorization': f'Bearer {tokenCanvas}',
}
params = {
    'per_page': 100,
}

def finn_rel(link_header):
    link_header_dict = {}
    for link in link_header.split(","):
        url, rel = link.strip().split(";")
        rel = rel.split('=')[1]
        link_header_dict[rel.strip().replace('"', '')] = url.strip().replace('<', '').replace('>', '')      # Kommentar: denne er med BERRE for å skape balanse i .org-fila: \"
    return link_header_dict

Først henter eg ut alle terminar frå Canvas:

In [14]:
url = "https://hvl.instructure.com/api/v1/accounts/1/terms"
termin = 353
dr_liste = []
hentmeir = True
while hentmeir:
    print(url)   # for kontrollen sin skuld
    respons = requests.get(url, headers=headers, params=params)
    if 200 <= respons.status_code < 300:
        data = respons.json()['enrollment_terms']
        # print(respons.headers)
        hentmeir = "next" in respons.headers['link']
        if hentmeir:
            url = finn_rel(respons.headers['link'])['next']
# Hent ut data (NB! dette vil variere frå endepunkt til endepunkt)
        dataliste = []
        for element in data:
            dataliste.append([element['id'], element['name']])
        df = pd.DataFrame(dataliste, columns=['id', 'name'])
        dr_liste.append(df)
    # hentmeir = False
# og etter at eg er ferdig å lese inn slår eg dei saman:
alledata = pd.concat((df for df in dr_liste if not df.empty), ignore_index=True)

https://hvl.instructure.com/api/v1/accounts/1/terms
https://hvl.instructure.com/api/v1/accounts/1/terms?page=2&per_page=100


Så lager eg to nye kolonner; ei for startsemester og ei for sluttsemester. Dei er på forma YYYYa og YYYYb der "a" er VÅR og "b" er HØST, lik at det er lettare å sjekke om ein termin ligg etter eller før eit gitt start- eller sluttsemester. 

In [17]:
temp = alledata.iloc[:, 1].astype(str).str.split("-", expand=True)

In [21]:
alledata['startsemester'] = temp[0].astype(str)
alledata['sluttsemester'] = temp[1].astype(str)

In [24]:
alledata['startsemester'] = alledata['startsemester'].str.replace(" VÅR", "a", regex=False)
alledata['sluttsemester'] = alledata['sluttsemester'].str.replace(" VÅR", "a", regex=False)
alledata['startsemester'] = alledata['startsemester'].str.replace(" HØST", "b", regex=False)
alledata['sluttsemester'] = alledata['sluttsemester'].str.replace(" HØST", "b", regex=False)

In [25]:
alledata

,id,name,startsemester,sluttsemester
0,1,Standardsemester,Standardsemester,None
1,2,2017 HØST,2017b,None
2,6,2017 HØST,2017b,None
3,7,2017 VÅR,2017a,None
4,8,2018 VÅR,2018a,None
...,...,...,...,...
121,362,2025VÅR,2025VÅR,None
122,363,ULK6 HØST,ULK6b,None
123,364,ULK7 HØST,ULK7b,None
124,365,2025 HØST-2027 VÅR,2025b,2027a


In [31]:
termin = "2025b"
terminar = alledata[(alledata['startsemester']<=termin)&(alledata['sluttsemester']>=termin)&(alledata['sluttsemester']!="None")]['id'].tolist()